In [9]:
import os
import numpy as np
import random as rng
import cv2
import json
import math
from scipy.misc import face
from scipy.signal.signaltools import wiener
import matplotlib.pyplot as plt

In [ ]:
info_data = []
with open('/home/data/strips_socrates/dataset_info.json') as json_file: 
    info_data = json.load(json_file) 

In [ ]:
STRIP_SIZE = 256
DENOISE_SIGMA = 5

In [4]:
def denoise_coefficient_list(coefficient_list, sigma):
    ll = coefficient_list[0]
    denoised_bands = [ll]
    for band, subband_coefficients in enumerate(coefficient_list[1 :]):
        denoised_bands.append([wiener(s.astype(np.float), sigma)
                               for s in subband_coefficients])
    return denoised_bands


In [5]:
def get_residual(grayscale_matrix):
    dyad_length = math.ceil(math.log(STRIP_SIZE, 2))
    ll_levels = 5
    wavelet_levels = dyad_length - ll_levels
    ll_size = 2 ** ll_levels
    coefficient_list = pywt.wavedec2(grayscale_matrix,
                                       'db8',
                                       level = int(wavelet_levels),
                                       mode = 'per')
    coefficient_list = denoise_coefficient_list(coefficient_list,
                                                  DENOISE_SIGMA)
    denoised_tile = pywt.waverec2(coefficient_list,
                                    'db8',
                                    mode = 'per')
    denoised_tile[denoised_tile > 255.0] = 255.0
    denoised_tile[denoised_tile < 0.0] = 0.0
    return (denoised_tile, grayscale_matrix - denoised_tile)


In [6]:
def get_final(denoised, residual):
    numerator = denoised * residual
    denominator = denoised * denoised
    return(numerator/denominator)

In [ ]:
def get_final_from_path(img_path):
    img = cv2.imread(img_path)
    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    den, res = get_residual(gimg)
    return (get_final(den, res))

In [33]:
def normalize_percentile_cut(mat, low, high, q_cut):
    low_q = np.percentile(mat, q_cut)
    high_q = np.percentile(mat, 100 - q_cut)
    mat[mat > high_q] = high_q
    mat[mat < low_q] = low_q
    cv2.normalize(mat, mat, low, high, cv2.NORM_MINMAX)
                  

In [38]:
mat_wo_outlier = np.random.rand(50, 50)
mat_w_outlier = np.random.rand(50, 50)
mat_w_outlier[0,0] = -100
mat_w_outlier[0,2] = 100

In [35]:
out = np.zeros((50,50))

In [40]:
normalize_percentile_cut(mat_w_outlier, 0, 256, 2 )

In [41]:
cv2.normalize(mat_w_outlier, out, 0, 256, cv2.NORM_MINMAX).astype(int)

array([[  0,   4, 256, ...,  21,  51, 163],
       [ 43, 247,  21, ..., 117, 160, 182],
       [ 33, 151,  24, ..., 192,  31,   0],
       ...,
       [184, 185,  64, ...,  48, 208,  29],
       [ 47,   1,  53, ...,  41,  53, 171],
       [241, 198, 126, ..., 143, 104, 220]])

In [ ]:
done = []

In [ ]:
info_data[0][0][0]

In [ ]:
done = []
def transform_dataset():
    stds = len(info_data)
    for i, std in enumerate(info_data):
        print(i, " of ", stds)
        for img in std:
            for strip in img[1]:
                fin = get_final_from_path(strip)
                nor
                fin = fin.astype(int)
                cv2.imwrite(strip, fin)

                
            

In [ ]:
transform_dataset()

In [ ]:
len(transformed)

In [ ]:
for im in transformed:
    plt.figure()
    plt.imshow(im)

In [ ]:
im = cv2.imread('/home/data/strips_socrates/101/Eurecom_101_picBG_001_0_0.PNG')
im2 = cv2.imread('/home/data/strips_socrates/101/Eurecom_101_picBG_001_0_1024.PNG')
plt.imshow(im)

In [ ]:
plt.imshow(im2)

In [ ]:

plt.imshow(im2-im)